In [ ]:
import xarray as xr 
import tams
import warnings
warnings.filterwarnings('ignore')

In [ ]:
ds = tams.load_example_mpas().rename({"tb":"ctt", "precip": "pr"}).isel(time=slice(1, 13))

c:\Users\Mr. Lettucce\.conda\envs\py312_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ds # Without rename

<xarray.Dataset> Size: 62MB
Dimensions:  (time: 127, lat: 180, lon: 341)
Coordinates:
  * time     (time) datetime64[ns] 1kB 2006-09-08T12:00:00 ... 2006-09-13T18:...
  * lat      (lat) float64 1kB -4.875 -4.625 -4.375 -4.125 ... 39.38 39.62 39.88
  * lon      (lon) float64 3kB 85.0 85.25 85.5 85.75 ... 169.2 169.5 169.8 170.0
Data variables:
    ctt      (time, lat, lon) float32 31MB nan nan nan nan ... 266.0 265.9 265.7
    pr       (time, lat, lon) float32 31MB ...

In [6]:
ds # With rename

<xarray.Dataset> Size: 6MB
Dimensions:  (time: 12, lat: 180, lon: 341)
Coordinates:
  * time     (time) datetime64[ns] 96B 2006-09-08T13:00:00 ... 2006-09-09
  * lat      (lat) float64 1kB -4.875 -4.625 -4.375 -4.125 ... 39.38 39.62 39.88
  * lon      (lon) float64 3kB 85.0 85.25 85.5 85.75 ... 169.2 169.5 169.8 170.0
Data variables:
    ctt      (time, lat, lon) float32 3MB 266.8 266.9 267.0 ... 262.9 261.2
    pr       (time, lat, lon) float32 3MB ...

In [7]:
# Let's remember that run() gives 3 output ds:

ce, mcs, mcs_sumary = tams.run(ds, parallel=True) # run needs a ds that MUST have cloud-top temperature and precip rate variables. The dims must be time, lat and lon. The former one must be in -180 --180 format.
# ce gives you a ds with the CEs identified. These CEs have an id  (tracking) and a mcs_class (classification). 

Starting `identify` 2026-02-13 17:58:01


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 tasks      | elapsed:   20.7s
[Parallel(n_jobs=-2)]: Done   3 out of  12 | elapsed:   21.0s remaining:  1.1min
[Parallel(n_jobs=-2)]: Done   5 out of  12 | elapsed:   21.1s remaining:   29.6s
[Parallel(n_jobs=-2)]: Done   7 out of  12 | elapsed:   21.3s remaining:   15.2s
[Parallel(n_jobs=-2)]: Done   9 out of  12 | elapsed:   22.1s remaining:    7.3s
[Parallel(n_jobs=-2)]: Done  12 out of  12 | elapsed:   22.3s finished


Starting `track` 2026-02-13 17:58:23
Starting `classify` 2026-02-13 17:58:25


c:\Users\Mr. Lettucce\.conda\envs\py312_env\Lib\site-packages\tams\core.py:824: FutureWarning: Calling ``EllipseModel()`` (without arguments) has been deprecated since version 0.26 and will be removed in version 2.2; see help for ``EllipseModel``.
  m = EllipseModel()
c:\Users\Mr. Lettucce\.conda\envs\py312_env\Lib\site-packages\tams\core.py:825: FutureWarning: `estimate` is deprecated since version 0.26 and will be removed in version 2.2. Please use `EllipseModel.from_estimate` class constructor instead.
  success = m.estimate(xy)
c:\Users\Mr. Lettucce\.conda\envs\py312_env\Lib\site-packages\tams\core.py:831: FutureWarning: `params` is deprecated since version 0.26 and will be removed in version 2.2. `params` attribute deprecated; use `center, axis_lengths, theta` attributes instead.
  _, _, xhw, yhw, _ = m.params
c:\Users\Mr. Lettucce\.conda\envs\py312_env\Lib\site-packages\tams\core.py:824: FutureWarning: Calling ``EllipseModel()`` (without arguments) has been deprecated since versi

Starting statistics calculations 2026-02-13 17:58:28
Starting CE aggregation (into MCS time series) 2026-02-13 17:58:28
Starting gridded data aggregation 2026-02-13 17:58:32


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-2)]: Done   3 out of  12 | elapsed:    3.1s remaining:    9.6s
[Parallel(n_jobs=-2)]: Done   5 out of  12 | elapsed:    3.2s remaining:    4.5s
[Parallel(n_jobs=-2)]: Done   7 out of  12 | elapsed:    3.7s remaining:    2.6s
[Parallel(n_jobs=-2)]: Done   9 out of  12 | elapsed:    5.5s remaining:    1.8s
[Parallel(n_jobs=-2)]: Done  12 out of  12 | elapsed:    5.6s finished


Computing stats for MCS summary dataset 2026-02-13 17:58:38
Done 2026-02-13 17:58:38


In [ ]:
ce.describe()

# some information commands:
# print(ce.info()) <> for some metadata like type of each variable available.
# ce.area_km2.describe() <> allows you see some statistic info, like mean, std and some percentiles. We can uninclude the area_km2 to see the values of the hole ds but that can be undesirable.


,area_km2,area219_km2,time,mcs_id
count,311.0,311.0,311,311.0
mean,322513.334878,140576.912541,2006-09-08 18:33:57.299035392,51.874598
min,10377.283677,4046.159504,2006-09-08 13:00:00,0.0
25%,45083.40339,12649.734483,2006-09-08 16:00:00,27.5
50%,158404.664835,52253.983371,2006-09-08 19:00:00,55.0
75%,415329.825091,189229.870501,2006-09-08 21:00:00,71.0
max,3252490.158201,1144420.930173,2006-09-09 00:00:00,103.0
std,454722.354567,200948.903397,NaN,27.878082


In [ ]:
mcs 

# This datasets shows that with each time, CEs with the same MCS id (tracking) are combined into one. Normally including pr and ctt. Each row corresponds to an MCS-time.
# The same information commands are available.

,time,cs235,cs219,nce,area_km2,area219_km2,mcs_id,mcs_class,mean_pr,std_pr,npixel,mean_pr219,std_pr219,mean_ctt219,std_ctt219,npixel219
0,2006-09-08 13:00:00,"MULTIPOLYGON (((167.5 9.49188, 167.75 9.58423,...","MULTIPOLYGON (((167.5 9.61125, 167.53439 9.625...",1,25634.1184,11798.035271,0,DSL,2.645099,0.927722,36,3.28087,0.533223,209.896072,4.906273,18
1,2006-09-08 13:00:00,"MULTIPOLYGON (((152.5 8.79886, 152.61238 8.875...","MULTIPOLYGON (((152.75 10.37642, 153 10.44407,...",1,79372.537834,48340.476814,1,DSL,1.910169,1.426157,106,2.547689,1.304625,215.991379,16.687454,64
2,2006-09-08 13:00:00,"MULTIPOLYGON (((148.75 8.23715, 149 8.27821, 1...","MULTIPOLYGON (((148.75 8.34082, 148.99758 8.37...",1,14036.478097,7223.353767,2,DSL,2.254961,0.982726,19,2.883151,0.706086,213.163315,4.442413,11
3,2006-09-08 13:00:00,"MULTIPOLYGON (((147.25 7.01224, 147.75 7.01643...","MULTIPOLYGON (((147.75 7.1996, 148 7.32651, 14...",1,22263.09014,6159.967885,3,DSL,2.019073,0.856571,29,2.915771,0.337212,214.483337,2.475573,9
4,2006-09-08 13:00:00,"MULTIPOLYGON (((138.25 15.00363, 138.5 15.0509...","MULTIPOLYGON (((138.25 15.10322, 138.33471 15....",1,10596.159009,5433.912228,4,DSL,1.95028,0.935729,15,2.571446,0.641791,211.823792,5.197964,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,2006-09-09 00:00:00,"MULTIPOLYGON (((126 16.04305, 126.25 16.10036,...","MULTIPOLYGON (((127.75 19.60362, 127.75 19.603...",4,963155.281892,326920.015309,98,DSL,0.951342,1.145551,1243,1.412011,1.488934,214.811386,6.743108,440
267,2006-09-09 00:00:00,"MULTIPOLYGON (((148.75 -2.21941, 149.5 -2.2141...","MULTIPOLYGON (((150.5 -0.67942, 150.55409 -0.6...",1,157397.412137,40993.731489,100,DSL,0.900998,0.597975,205,1.25952,0.619944,217.582062,4.004134,55
268,2006-09-09 00:00:00,"MULTIPOLYGON (((97 22.14142, 97.25 22.17086, 9...","MULTIPOLYGON (((98.25 22.91528, 98.28253 23.12...",1,780231.600791,309818.201344,101,DSL,1.046286,2.042402,1002,1.905522,2.855528,216.48819,13.512681,393
269,2006-09-09 00:00:00,"MULTIPOLYGON (((168.75 19.50717, 170 19.75475,...","MULTIPOLYGON (((169 22.58414, 169.03442 22.625...",1,724340.022743,12713.449719,102,DSL,1.05563,0.930567,931,1.742241,1.036302,218.198257,3.076819,24


In [19]:
mcs_sumary.describe()

# In this case, we don't include shapes but the initial and final point (i.e. Tracking)
# All of the information commands area available.

,first_time,last_time,duration,mcs_id,mean_mean_pr,mean_mean_pr219,mean_mean_ctt219,mean_std_ctt219,mean_area_km2,mean_area219_km2,mean_nce,distance_km
count,104,104,104,104.0,104.0,104.0,104.0,104.0,104.0,104.0,104.0,104.0
mean,2006-09-08 16:23:04.615384576,2006-09-08 17:59:25.384615424,0 days 02:36:20.769230769,51.5,1.156704,1.61739,213.8578,7.218748,208890.559483,83853.57485,1.066083,58.185853
min,2006-09-08 13:00:00,2006-09-08 13:00:00,0 days 01:00:00,0.0,0.187491,0.456384,205.555122,0.343413,10596.159009,4474.283814,1.0,0.0
25%,2006-09-08 14:00:00,2006-09-08 15:00:00,0 days 01:00:00,25.75,0.770805,1.092501,211.329475,4.331841,28062.533802,9114.303342,1.0,0.0
50%,2006-09-08 15:30:00,2006-09-08 17:00:00,0 days 01:00:00,51.5,1.059372,1.393241,214.508239,6.731265,65245.319034,18828.285517,1.0,0.0
75%,2006-09-08 18:00:00,2006-09-08 21:00:00,0 days 03:00:00,77.25,1.45971,1.95973,216.562508,9.324257,207862.252969,76796.45252,1.0,53.377108
max,2006-09-09 00:00:00,2006-09-09 00:00:00,0 days 10:00:00,103.0,3.069273,6.575438,222.817734,20.814117,2000734.41701,812164.938688,2.666667,750.167907
std,NaN,NaN,0 days 02:30:49.401384730,30.166206,0.547035,0.857315,3.514747,4.020769,351372.669445,150007.990804,0.226198,117.328995
